
**Imports libraries**

**Opens HDF5 file into file handler.**

Creates megaList and trialList

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import math as math
import scipy.interpolate as interp

file = h5py.File('redacted', 'r')

#ls - list of files base group names
ls = list(file.keys()) 
print('Keys: \n',ls,'\n')

trialList = [] #For r1 data Trial Number
megaList = [] #For r2 and r1 data


**Adds all individual arrays to an MEGA 2D Array** where each index of MEGA is an numpy array (of either r1 or r2 data).

Because of order of operation, every 3 indexes of Mega array are associated data, such that it's 2 r2 & 1 r1 arrays. Every 3rd index starting at 2 is r1 data (E.g indexes 2, 5, 8, 11, etc) - so:

megaList[**0**] = r2, 

megaList[**1**] = r2, 

megaList[**2**] = r1, repeat etc

In [ ]:
def visitor_func(name, node):
    if isinstance(node, h5py.Dataset):
         dataArray = np.array(node) #  Converts r2/r1 data into numpy array

         #If r2 data: append to megaList. || If r1 data, reformat then append to megaList
         #r1 data is column-wise and needs to be reformated to row-wise
         match dataArray.shape[0]:
             case 1: reformatedDataArray = dataArray[0]
             case _: reformatedDataArray = np.reshape(dataArray, dataArray.size); trialList.append(node.name)
         
         megaList.append(reformatedDataArray); #  all arrays are appended to a megaList - per every 3 indexes are associated data starting from 0.
    #else: node is a group: does nothing.

for s in file.keys():
    for x in file.get(s):
        folder = file.get('{}/{}'.format(s,x))
        folder.visititems(visitor_func)


Attempts to Normalize the data before plotting. Uses MinMax Normalization.

r1 values range from -x000 to +x000 (±x000)

r2 values range from x to 0.xx

Current issue:

1. minmax normalization isn't fixing r1 negative values

2. r2 data is x-axis short, about half the amount of r1 data points. Need a solution to extend r2.

In [ ]:
def ecgAdjust(r2, r1):
    arr2_interp = interp.interp1d(np.arange(r2.size),r2,)
    return arr2_interp(np.linspace(0,r2.size-1,r1.size))
    

for x in range(2, len(megaList), 3): #X is always next r1 value
    plt.figure()

    #MinMax Normalization:
    r1 = (megaList[x] - np.min(megaList[x])) / (np.max(megaList[x]) - np.min(megaList[x])) 
    r2 = (megaList[x-1] - np.min(megaList[x-1])) / (np.max(megaList[x-1]) - np.min(megaList[x-1]))

    r2 = ecgAdjust(r2, r1)
    
    plt.plot(r2, label='redacted1')
    plt.plot(r1, label='redacted2')
    plt.legend()
    plt.title("Trial {}".format(trialList[int((x - 2) / 3)]))
    plt.xlabel("Time")
    plt.ylabel("Amplitude")

plt.show 

In [ ]:
plt.close()
file.close() #probably important